<a href="https://colab.research.google.com/github/hradawski/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/HernanRadawski_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hernán Radawski <br>
Link: https://colab.research.google.com/drive/1meCNwpfCue2dkH0dXwpd-1GLPmikU6E7 <br>
Actividad Guiada 3

In [0]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f57bd0c4128>)

In [0]:
!pip install tsplib95
import tsplib95
import random
problem= tsplib95.load_problem(file)

In [0]:
Nodos = list(problem.get_nodes())
Aristas= list(problem.get_edges())

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion  
crear_solucion(Nodos)

[0,
 1,
 13,
 12,
 41,
 17,
 10,
 35,
 19,
 33,
 39,
 20,
 30,
 3,
 2,
 21,
 9,
 14,
 32,
 28,
 31,
 25,
 5,
 7,
 22,
 36,
 24,
 38,
 37,
 6,
 34,
 15,
 16,
 11,
 29,
 40,
 23,
 26,
 8,
 27,
 4,
 18]

In [0]:
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)
distancia(0,1,problem)

15

In [0]:
#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

solucion = crear_solucion(Nodos)
distancia_total(solucion, problem)

4451

## Busqueda Aleatoria

In [0]:
def busqueda_aleatoria(problem, N): 
    mejor_solucion = crear_solucion(Nodos)
    mejor_valor = distancia_total(mejor_solucion, problem)
    for i in range(N):
        solucion = crear_solucion(Nodos)
        valor = distancia_total(solucion, problem)
        if valor < mejor_valor:
            mejor_solucion = solucion 
            mejor_valor = valor
    print('La mejor solucion es ', mejor_valor, mejor_solucion)

busqueda_aleatoria(problem, 20000)


La mejor solucion es  3740 [0, 35, 36, 33, 20, 7, 24, 8, 30, 18, 6, 27, 41, 22, 32, 23, 34, 31, 15, 14, 13, 17, 10, 12, 21, 40, 38, 29, 1, 25, 11, 28, 3, 4, 5, 26, 16, 19, 9, 2, 37, 39]


In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
    mejor_solucion = []
    mejor_distancia = 10e100
    for i in range(1,len(solucion)-1):
        for j in range(i+1, len(solucion)):
            vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
            #print(vecina)
            distancia_vecina = distancia_total(vecina, problem)
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina
    #print(mejor_distancia, mejor_solucion)
    return mejor_solucion
      
solucion = crear_solucion(Nodos)
#print("solucion ", solucion)
mejor_solucion = genera_vecina(solucion)
print(mejor_solucion, distancia_total(mejor_solucion, problem))

[0, 35, 34, 17, 8, 30, 27, 38, 37, 3, 13, 25, 14, 7, 15, 4, 18, 36, 22, 21, 40, 39, 10, 12, 32, 9, 33, 28, 26, 23, 19, 1, 31, 2, 6, 16, 5, 11, 20, 29, 41, 24] 4271


In [0]:
#####################################################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
    mejor_solucion = []
    mejor_distancia = 10e100
  
    solucion_referencia = crear_solucion(Nodos)
  
    for i in range(N):
        vecina = genera_vecina(solucion_referencia)
        distancia_vecina = distancia_total(vecina, problem)
        if distancia_vecina <= mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina

        solucion_referencia = vecina
        
    print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

busqueda_local(problem, 500)    

La mejor solución encontrada es  1652 [0, 6, 5, 26, 18, 13, 19, 14, 16, 15, 37, 17, 36, 35, 31, 10, 23, 41, 25, 11, 12, 3, 32, 34, 33, 20, 7, 1, 4, 8, 9, 21, 40, 24, 39, 22, 38, 30, 29, 28, 2, 27]


In [51]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina
solucion= crear_solucion(Nodos)
print(solucion, "\n", genera_vecina_aleatorio(genera_vecina_aleatorio(solucion)))

[0, 15, 19, 6, 10, 28, 40, 4, 27, 38, 30, 11, 7, 17, 36, 9, 16, 37, 20, 1, 23, 35, 32, 39, 21, 14, 5, 8, 29, 26, 34, 22, 24, 41, 18, 12, 13, 25, 31, 2, 33, 3] 
 [0, 15, 19, 6, 10, 28, 40, 4, 27, 38, 30, 12, 7, 17, 36, 9, 16, 37, 20, 1, 23, 35, 32, 39, 21, 14, 5, 8, 33, 26, 34, 22, 24, 41, 18, 11, 13, 25, 31, 2, 29, 3]


In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
import math
def probabilidad(T,d):
    if random.random() <= math.exp(-1*d / T)   :
        return True
    else:
        return False


def bajar_temperatura(T):
    return T*.9999

In [0]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')

    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    TEMPERATURA = bajar_temperatura(TEMPERATURA)
  
  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=1000)

	****Encontrada mejor solucion: 5491 [0, 10, 36, 30, 29, 26, 40, 37, 15, 5, 21, 14, 6, 33, 4, 20, 11, 12, 34, 24, 35, 2, 25, 1, 23, 28, 18, 17, 27, 8, 19, 9, 39, 31, 41, 7, 3, 16, 22, 13, 38, 32] 

	****Encontrada mejor solucion: 5131 [0, 10, 36, 30, 29, 26, 40, 37, 15, 5, 16, 14, 6, 33, 4, 20, 11, 27, 34, 24, 35, 2, 25, 1, 23, 28, 18, 17, 12, 8, 19, 9, 39, 31, 41, 7, 3, 21, 22, 13, 38, 32] 

	****Encontrada mejor solucion: 4889 [0, 10, 36, 30, 29, 26, 3, 37, 15, 5, 16, 14, 6, 33, 4, 20, 11, 27, 34, 24, 35, 2, 25, 1, 23, 28, 18, 17, 12, 8, 19, 9, 39, 31, 41, 7, 40, 21, 22, 13, 38, 32] 

	****Encontrada mejor solucion: 4887 [0, 29, 25, 23, 35, 30, 24, 22, 19, 36, 26, 4, 33, 20, 15, 5, 3, 12, 41, 8, 17, 2, 6, 11, 32, 10, 31, 38, 37, 34, 14, 16, 39, 18, 27, 7, 40, 1, 21, 13, 9, 28] 

	****Encontrada mejor solucion: 4815 [0, 29, 25, 23, 35, 30, 24, 22, 19, 36, 26, 39, 33, 20, 15, 5, 3, 12, 41, 8, 17, 2, 6, 11, 32, 10, 31, 38, 37, 34, 14, 16, 4, 18, 27, 7, 40, 1, 21, 13, 9, 28] 

	****Encon

In [0]:
# Colonia de Hormigas 

def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
    Nodos = list(problem.get_nodes())
    #print("lista ",  list(set(range(1,len(Nodos))) - set(H) ), " H : ", H   )
    eleccion = random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )
    #print(eleccion)
    return eleccion


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
    for i in range(len(H)-1):
        T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
    return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
    T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
    return T

In [0]:
# Colonia de Hormigas 

def hormigas(problem, N) :
    #problem = datos del problema
    #N = Número de agentes(hormigas)

    #Nodos
    Nodos = list(problem.get_nodes())
    #Aristas
    Aristas = list(problem.get_edges()) 

    #Inicializa las aristas con una cantidad inicial de feromonas:1
    # Mejora, inicializarlo con un valor proporsional a la distancia, reforzando las mas cortas
    T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]

    #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
    Hormiga = [[0] for _ in range(N)]
    
    #Recorre cada agente construyendo la solución
    for h in range(N) :
        #print("\nAgente:", h)
        #Para cada agente se construye un camino
        for i in range(len(Nodos)-1) :
            
            #Elige el siguiente nodo
            Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )

            Hormiga[h].append(Nuevo_Nodo)     

        #Incrementa feromonas en esa arista 
        T = Incrementa_Feromona(problem, T, Hormiga[h] )
        #print("Feromonas(1)", T)

        #Evapora Feromonas  
        T = Evaporar_Feromonas(T)
        #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
    #print ("Hormiga", Hormiga)
    mejor_solucion = []
    mejor_distancia = 10e100
    for h in range(N) :
        distancia_actual = distancia_total(Hormiga[h], problem)
        if distancia_actual < mejor_distancia:
            mejor_solucion = Hormiga[h]
            mejor_distancia =distancia_actual
    print(mejor_solucion)
    print(mejor_distancia)

In [0]:
hormigas(problem, 1000)

[0, 35, 15, 10, 12, 17, 19, 14, 16, 13, 27, 9, 23, 41, 18, 25, 26, 20, 22, 4, 3, 30, 40, 32, 37, 7, 33, 5, 34, 28, 8, 24, 21, 39, 2, 1, 6, 11, 36, 31, 38, 29]
3709


In [0]:
print([[ 1 for _ in range(10) ] for _ in range(10)])